# Importy

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

import csv

# Ładowanie danych

Ładowanie danych z dysku

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Koło/'
df_train = pd.read_csv(path + 'train_1.csv')
df_test = pd.read_csv(path + 'test_1.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95390 entries, 0 to 95389
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Cena             95390 non-null  int64  
 1   Stan             95390 non-null  object 
 2   Marka_pojazdu    95390 non-null  object 
 3   Model_pojazdu    95390 non-null  object 
 4   Rok_produkcji    95390 non-null  int64  
 5   Przebieg_km      95390 non-null  float64
 6   Moc_KM           95390 non-null  int64  
 7   Pojemnosc_cm3    95390 non-null  int64  
 8   Rodzaj_paliwa    95390 non-null  object 
 9   Naped            95390 non-null  object 
 10  Skrzynia_biegow  95390 non-null  object 
 11  Typ_nadwozia     95390 non-null  object 
 12  Liczba_drzwi     95390 non-null  int64  
 13  Kolor            95390 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 10.2+ MB


# Enkodowanie

Enkodowanie w celu pozbycia się wartości nienumerycznych - poprzez częstotliwość danych w całym zbiorze

### Training

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Stan'])

In [ ]:
marka_frequency = df_train.groupby('Marka_pojazdu').size()/len(df_train)
df_train.loc[:, 'Marka_pojazdu_encoded'] = df_train['Marka_pojazdu'].map(marka_frequency)
df_train.drop(['Marka_pojazdu'], axis=1, inplace=True)

In [ ]:
model_frequency = df_train.groupby('Model_pojazdu').size()/len(df_train)
df_train.loc[:, 'Model_pojazdu_encoded'] = df_train['Model_pojazdu'].map(model_frequency)
df_train.drop(['Model_pojazdu'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_train.groupby('Rodzaj_paliwa').size()/len(df_train)
df_train.loc[:, 'Rodzaj_paliwa_encoded'] = df_train['Rodzaj_paliwa'].map(nadwozie_frequency)
df_train.drop(['Rodzaj_paliwa'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_train.groupby('Naped').size()/len(df_train)
df_train.loc[:, 'Naped_encoded'] = df_train['Naped'].map(nadwozie_frequency)
df_train.drop(['Naped'], axis=1, inplace=True)

In [ ]:
skrzynia_frequency = df_train.groupby('Skrzynia_biegow').size()/len(df_train)
df_train.loc[:, 'Skrzynia_biegow_encoded'] = df_train['Skrzynia_biegow'].map(skrzynia_frequency)
df_train.drop(['Skrzynia_biegow'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_train.groupby('Typ_nadwozia').size()/len(df_train)
df_train.loc[:, 'Typ_nadwozia_encoded'] = df_train['Typ_nadwozia'].map(nadwozie_frequency)
df_train.drop(['Typ_nadwozia'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_train.groupby('Kolor').size()/len(df_train)
df_train.loc[:, 'Kolor_encoded'] = df_train['Kolor'].map(nadwozie_frequency)
df_train.drop(['Kolor'], axis=1, inplace=True)

### Test

In [ ]:
df_test = pd.get_dummies(df_test, columns=['Stan'])

In [ ]:
marka_frequency = df_test.groupby('Marka_pojazdu').size()/len(df_test)
df_test.loc[:, 'Marka_pojazdu_encoded'] = df_test['Marka_pojazdu'].map(marka_frequency)
df_test.drop(['Marka_pojazdu'], axis=1, inplace=True)

In [ ]:
model_frequency = df_test.groupby('Model_pojazdu').size()/len(df_test)
df_test.loc[:, 'Model_pojazdu_encoded'] = df_test['Model_pojazdu'].map(model_frequency)
df_test.drop(['Model_pojazdu'], axis=1, inplace=True)

In [ ]:
skrzynia_frequency = df_test.groupby('Rodzaj_paliwa').size()/len(df_test)
df_test.loc[:, 'Rodzaj_paliwa_encoded'] = df_test['Rodzaj_paliwa'].map(skrzynia_frequency)
df_test.drop(['Rodzaj_paliwa'], axis=1, inplace=True)

In [ ]:
skrzynia_frequency = df_test.groupby('Naped').size()/len(df_test)
df_test.loc[:, 'Naped_encoded'] = df_test['Naped'].map(skrzynia_frequency)
df_test.drop(['Naped'], axis=1, inplace=True)

In [ ]:
skrzynia_frequency = df_test.groupby('Skrzynia_biegow').size()/len(df_test)
df_test.loc[:, 'Skrzynia_biegow_encoded'] = df_test['Skrzynia_biegow'].map(skrzynia_frequency)
df_test.drop(['Skrzynia_biegow'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_test.groupby('Typ_nadwozia').size()/len(df_test)
df_test.loc[:, 'Typ_nadwozia_encoded'] = df_test['Typ_nadwozia'].map(nadwozie_frequency)
df_test.drop(['Typ_nadwozia'], axis=1, inplace=True)

In [ ]:
nadwozie_frequency = df_test.groupby('Kolor').size()/len(df_test)
df_test.loc[:, 'Kolor_encoded'] = df_test['Kolor'].map(nadwozie_frequency)
df_test.drop(['Kolor'], axis=1, inplace=True)

Wypełnianie medianą po zakodowaniu

In [ ]:
df_test['Naped_encoded'] = df_test['Naped_encoded'].fillna(df_test['Naped_encoded'].median())
df_test['Skrzynia_biegow_encoded'] = df_test['Skrzynia_biegow_encoded'].fillna(df_test['Skrzynia_biegow_encoded'].median())
df_test['Liczba_drzwi'] = df_test['Liczba_drzwi'].fillna(df_test['Liczba_drzwi'].median())

In [ ]:
df_test.isnull().sum()

,0
ID,0
Rok_produkcji,0
Przebieg_km,0
Moc_KM,0
Pojemnosc_cm3,0
Liczba_drzwi,0
Stan_New,0
Stan_Used,0
Marka_pojazdu_encoded,0
Model_pojazdu_encoded,0


# Zbiory

### Podział X i y

Podział na zbiór cech X i zbiór etykiet y

In [ ]:
y_train_org = df_train['Cena']
X_train_org = df_train.drop(['Cena'], axis=1)

In [ ]:
X_test = df_test

### Trening i walidacja

Podział na zbiór treningowy i walidacyjny

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_org, y_train_org, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_val = scaler.fit_transform(X_val)
X_scaled_test = scaler.fit_transform(X_test)

pca = PCA()
pca.fit(X_scaled_train)

cumulative_variance = pca.explained_variance_ratio_.cumsum()

threshold = 0.90
n_components = (cumulative_variance <= threshold).sum() + 1
print(f"Liczba komponentów wyjaśniających co najmniej {threshold * 100}% wariancji: {n_components}")

Liczba komponentów wyjaśniających co najmniej 90.0% wariancji: 9


In [ ]:
pca = PCA(n_components=n_components)
X_pca_train = pca.fit_transform(X_scaled_train)
X_pca_val = pca.fit_transform(X_scaled_val)
X_pca_test = pca.fit_transform(X_scaled_test)

# Modele

### Random forest

Dopasowanie parametrów

In [ ]:
random_forest = RandomForestRegressor()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf_random = RandomizedSearchCV(estimator = random_forest, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)
rf_random.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=  46.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=  46.0s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=  44.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=  44.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=  44.8s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time= 1.3min
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time= 1.3min
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=1,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [ ]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=25, max_features='sqrt', n_estimators=1000)

In [ ]:
print(rf_random.best_estimator_.min_samples_split)
print(rf_random.best_estimator_.min_samples_leaf)

2
1


Gotowy model

In [ ]:
random_forest = RandomForestRegressor(max_depth=25, n_estimators=1000, max_features='sqrt', min_samples_leaf=1, min_samples_split=2)

In [ ]:
random_forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=25, max_features='sqrt', n_estimators=1000)

Ewaluacja na zbiorze walidacynym

In [ ]:
y_pred = random_forest.predict(X_val)

mae_val = mean_absolute_error(y_val, y_pred)
mse_val = mean_squared_error(y_val, y_pred)
rmse_val = np.sqrt(mse_val)

print(f"Walidacja Decision Tree - MAE: {mae_val:.4f}, MSE: {mse_val:.4f}, RMSE: {rmse_val:.4f}")

Walidacja Decision Tree - MAE: 8536.9813, MSE: 371073320.7025, RMSE: 19263.2635


### XGBoost

Model

In [ ]:
xgboost = xgb.XGBRegressor(base_score=0.5, gamma=0.08, learning_rate=0.075,
                       max_depth=7, min_child_weight=1, n_estimators=560,
                       reg_alpha=0, reg_lambda=1,
                       subsample=0.75)

xgboost.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.08, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.075, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=560, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Ewaluacja na zbiorze walidacyjnym

In [ ]:
y_pred = xgboost.predict(X_val)
mae_val = mean_absolute_error(y_val, y_pred)
mse_val = mean_squared_error(y_val, y_pred)
rmse_val = np.sqrt(mse_val)

print(f"Walidacja Decision Tree - MAE: {mae_val:.4f}, MSE: {mse_val:.4f}, RMSE: {rmse_val:.4f}")

Walidacja Decision Tree - MAE: 8341.2588, MSE: 344846560.0000, RMSE: 18570.0447


Predykcja zbioru testowego

In [ ]:
ID_col_test = X_test['ID']
X_test = X_test.drop(['ID'], axis=1)
y_pred = xgboost.predict(X_test)
X_test['ID'] = ID_col_test

In [ ]:
with open('submission.csv', mode='w', newline='') as plik:
    writer = csv.writer(plik)
    writer.writerow(['ID', 'Cena'])
    for id, cena in zip(X_test['ID'], y_pred):
        writer.writerow([id, cena])